In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
from torch_geometric.transforms import RandomLinkSplit

In [44]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, aggr='mean'):
        super(Encoder, self).__init__()
        self.conv1 = pyg_nn.SAGEConv(in_channels, 2 * out_channels, aggr=aggr) # Eventually change layer type
        self.batch1 = pyg_nn.BatchNorm(2 * out_channels)
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = pyg_nn.SAGEConv(2 * out_channels, out_channels,  aggr=aggr) # Eventually change layer type
        self.batch2 = pyg_nn.BatchNorm(out_channels)

    def forward(self, x, edge_index): # Modify to adapt to the dataset complexity
        x = self.conv1(x, edge_index)
        x = self.batch1(x)
        x = self.drop1(x)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.batch2(x)

        return x
    
class Decoder(nn.Module):
    def __init__(self, in_channels):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(in_channels, 1)

    def forward(self, z, edge_index):
        # By default, it computes the inner product of the node embeddings
        adj = torch.matmul(z[edge_index[0]], z[edge_index[1]].t())
        return adj[edge_index[0], edge_index[1]]
    
# Defining traing and evaluation functions
def train(epoch):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_data.edge_index.to(device))
    loss = model.recon_loss(z, train_data.pos_edge_label_index.to(device))
    loss.backward()
    optimizer.step()

def test(pos_edge_index, neg_edge_index):
    model.eval()
    # Encode the node features to get the embeddings
    with torch.no_grad():
        z = model.encode(x, train_data.edge_index.to(device))
    
    # Compute the reconstruction loss
    return model.test(z, pos_edge_index.to(device), neg_edge_index.to(device))
    

In [45]:
# Model definition
import os
LATENT_DIM = 32
data_path = os.path.join('..', 'data', 'graph_data.pt')
dataset = torch.load(data_path, weights_only=False)
data = dataset['data']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Instantiate the customized encoder in GAE model
transform = RandomLinkSplit(is_undirected=True, split_labels=True,add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

model = pyg_nn.GAE(Encoder(data.num_features, LATENT_DIM)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x = train_data.x.to(device)

Using device: cuda


In [46]:
# Training loop
for epoch in range(1, 1000):
    loss = train(epoch)
    # AUC tells how well the model is able to distinguish real nodes from fake ones
    # AP - Average Precision is the complementary metric of AUC 
    auc, ap = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)

    if epoch % 10 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 010, AUC: 0.5122, AP: 0.4666
Epoch: 020, AUC: 0.4856, AP: 0.4499
Epoch: 030, AUC: 0.6260, AP: 0.5633
Epoch: 040, AUC: 0.7563, AP: 0.6762
Epoch: 050, AUC: 0.8088, AP: 0.7294
Epoch: 060, AUC: 0.8189, AP: 0.7363
Epoch: 070, AUC: 0.8245, AP: 0.7414
Epoch: 080, AUC: 0.8255, AP: 0.7418
Epoch: 090, AUC: 0.8146, AP: 0.7324
Epoch: 100, AUC: 0.8187, AP: 0.7369
Epoch: 110, AUC: 0.7470, AP: 0.6829
Epoch: 120, AUC: 0.8014, AP: 0.7616
Epoch: 130, AUC: 0.7782, AP: 0.7498
Epoch: 140, AUC: 0.8467, AP: 0.8188
Epoch: 150, AUC: 0.8514, AP: 0.8226
Epoch: 160, AUC: 0.8471, AP: 0.8198
Epoch: 170, AUC: 0.8380, AP: 0.8114
Epoch: 180, AUC: 0.8512, AP: 0.8234
Epoch: 190, AUC: 0.8573, AP: 0.8294
Epoch: 200, AUC: 0.8387, AP: 0.8123
Epoch: 210, AUC: 0.8605, AP: 0.8321
Epoch: 220, AUC: 0.8563, AP: 0.8273
Epoch: 230, AUC: 0.8604, AP: 0.8320
Epoch: 240, AUC: 0.8529, AP: 0.8211
Epoch: 250, AUC: 0.8539, AP: 0.8229
Epoch: 260, AUC: 0.8495, AP: 0.8178
Epoch: 270, AUC: 0.8462, AP: 0.8158
Epoch: 280, AUC: 0.8636, AP:

In [47]:
model.eval()
z = model.encode(x, train_data.edge_index.to(device))
emb_sample = z[0]
print(f'Number of nodes: {z.shape[0]}')
print(f'Number of edges: {train_data.edge_index.shape[1]}')
print(f'Embedding sample: {z}')

Number of nodes: 4877
Number of edges: 20618
Embedding sample: tensor([[ 0.2487,  0.5004,  0.2333,  ...,  0.1051,  0.2563,  0.2359],
        [-0.0426,  0.0289,  0.5784,  ...,  0.5255, -0.4166,  0.0175],
        [-0.0225,  0.3638,  0.3054,  ...,  0.4168, -0.0886,  0.1373],
        ...,
        [ 5.0085,  2.5669, -3.1004,  ...,  2.7502,  1.9506, -2.4619],
        [ 5.0036,  2.5650, -3.0984,  ...,  2.7469,  1.9492, -2.4614],
        [ 5.0097,  2.5674, -3.1009,  ...,  2.7510,  1.9510, -2.4620]],
       device='cuda:0', grad_fn=<NativeBatchNormBackward0>)
